In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json
from time import sleep

In [2]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    if n_attempts == 0:
        print("error with ", url)
        return "error"
    else:
        try:
            page = requests.get(url)
            return page
        except:
            sleep(t_sleep)
            return get_page(url, n_attempts-1, t_sleep, **kwargs)

In [3]:
link_list = []
for i in range(1,251):
    if i == 1:
        str_page = 'https://shop.relod.ru/catalog-products/4577/?view=list&sort=PROPERTY_RATING&order=desc'
    else:
        str_page = 'https://shop.relod.ru/catalog-products/4577/?view=list&sort=PROPERTY_RATING&order=desc&PAGEN_1='+str(i)
    
    page = get_page(str_page)
    try:
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'lxml')
            links = soup.find_all('div', class_="bxr-element-name hidden-xl hidden-lg hidden-md")
            for l in links:
                link_list.append(l.find('a').get('href'))
        else:
            print("code is not 200 ", str_page)
    except:
        pass

In [4]:
len(set(link_list))

4998

In [4]:
if len(set(link_list)) < 5000:
    str_page = 'https://shop.relod.ru/catalog-products/4577/?view=list&sort=PROPERTY_RATING&order=desc&PAGEN_1=251'
    page = get_page(str_page)
    #если в page.status_code возниклнет ошибка, значит get_page вернула строку "error" и уже вывела сообщение об этом
    try:
        if page.status_code == 200:
            soup = BeautifulSoup(page.text, 'lxml')
            links = soup.find_all('div', class_="bxr-element-name hidden-xl hidden-lg hidden-md")
            for l in links:
                if len(set(link_list)) < 5000:
                    link_list.append(l.find('a').get('href'))
                else:
                    break
    except:
        pass

In [5]:
len(set(link_list))

5000

In [6]:
len(link_list)

5002

In [10]:
from  more_itertools import unique_everseen
link_list = list(unique_everseen(link_list))
len(link_list)

5000

In [12]:
def process_page(url):
    arg_list = {}
    arg_list['url'] = 'https://shop.relod.ru' + url
    page = get_page(arg_list['url'])
    #если в page.status_code возниклнет ошибка, значит get_page вернула строку "error" и уже вывела сообщение об этом
    try:
        if page.status_code != 200:
            print('status code ',page.status_code, url )
            return {}  
    except:
        return {}       
    
    #name
    soup = BeautifulSoup(page.text, 'lxml')
    name = soup.find('h1', itemprop="name")
    arg_list['Название'] = name.text
    
    #rating
    rating = soup.find('div', class_='bxr-rating-detail')
    if rating.find(attrs={"itemprop": "ratingCount"}):
        arg_list['Число голосов'] = rating.find(attrs={"itemprop": "ratingCount"}).get('content')
        arg_list["Оценка"] = rating.find(attrs={"itemprop": "ratingValue"}).get('content')
    
    #price   
    price = soup.find('div', class_= 'bxr-right-col-detail bxr-detail-col')
    arg_list['Наличие'] = price.find(itemprop="availability").text
    if price.find(itemprop='price'): 
        arg_list['Цена'] = price.find(itemprop='price').get('content')
    pric = soup.find_all('script')
    for p in pric:
        if re.findall(r'PRICE_FORMAT',p.text):
                     arg_list['Цена (скидка)'] = re.findall(r'PRICE_FORMAT[^\n<]+',p.text)[0][16:]
    
    #markers
    a = re.findall(r'\b[^\n]+',soup.find('div', class_='bxr-ribbon-marker-vertical').text)
    if a:
        arg_list['Метки'] = a
   
    #pictures   
    if soup.find_all('img', id= lambda s:s):
        arg_list['Иллюстрации'] = list(map(lambda s: 'https'+s.get('data-src'), soup.find_all('img', id= lambda s:s)))
    
    #description
    arg_list['Описание'] = ''.join(re.findall(r'[^\n\r\t]+',soup.find('div', class_='bxr-detail').text))
    table = soup.find('table', class_="bxr-props-table")
    col = table.find_all('td', class_="bxr-props-name")
    val = table.find_all('td', class_="bxr-props-data")
    for c,v in zip(col,val):
        arg_list[c.text] = ' '.join(re.findall(r'[^\s]+', v.find(itemprop="value").text))
    return arg_list


In [13]:
import gzip
import json
import codecs
from tqdm import tqdm

from multiprocessing.dummy import Pool, Queue

queue = Queue()   # очередь ссылок на книги
for l in link_list:
    queue.put(l)

def process_page_wrapper(i):
    with gzip.open('part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            try:
                p = queue.get()
                record = process_page(p)
                record_str = json.dumps(record, ensure_ascii=False)
                print(record_str, file=f_json)
                     # счетчик должен атомарно обновиться
                with lock:
                    pbar.update(1)
            except:
                pass

with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

  2%|▏         | 120/5000 [01:54<1:56:52,  1.44s/it]

status code  404 /catalog-products/woes_of_the_true_policeman/


 28%|██▊       | 1411/5000 [25:21<55:59,  1.07it/s]  

status code  404 /catalog-products/the_two_week_wait/


 49%|████▉     | 2473/5000 [44:39<32:10,  1.31it/s]  

...

 84%|████████▍ | 4216/5000 [1:04:35<07:01,  1.86it/s]

 404 /catalog-products/the_dark_tower/


 98%|█████████▊| 4884/5000 [1:11:48<00:59,  1.94it/s]

status code  404 /catalog-products/the_angel_esmeralda_nine_stories/


100%|██████████| 5000/5000 [1:59:24<00:00,  1.43s/it]    


In [32]:
import jsonlines

def records_reader():
    for i in range(8):
        with gzip.open('part_{:05d}.jsonl.gz'.format(i), "rb") as f:
            for d in jsonlines.Reader(f):
                yield d
            
            
df = pd.DataFrame(records_reader())   
df.head(567)

,url,Название,Число голосов,Оценка,Наличие,Цена,Метки,Иллюстрации,Описание,ISBN,...,Страна происхождения,Рейтинг,Вес (грамм),Размер (мм),Цена (скидка),Читательская аудитория,Носитель,Тип продукта,Издание,Обзор/Ролик
0,https://shop.relod.ru/catalog-products/wives_a...,Wives and Daughters,2,5,Под заказ,139,[Sale],[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,"Wives and Daughters, Elizabeth Gaskell's last...",9780199538263,...,Соединенное Королевство,3.44,530,195(д) х 128(ш) х 36(в),NaN,NaN,NaN,NaN,NaN,NaN
1,https://shop.relod.ru/catalog-products/the_fel...,The Fellowship of the Ring,3,3.33,Под заказ,1113,NaN,[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,"Continuing the story begun in The Hobbit, this...",9780007488315,...,NaN,3.17,310,198(д) х 129(ш) х 29(в),NaN,NaN,NaN,NaN,NaN,NaN
2,https://shop.relod.ru/catalog-products/zoo_tails/,Zoo Tails,NaN,NaN,Под заказ,989,NaN,[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,Oliver Graham-Jones recalls his time as the ch...,9780857502605,...,NaN,NaN,146,199(д) х 128(ш) х 24(в),NaN,NaN,NaN,NaN,NaN,NaN
3,https://shop.relod.ru/catalog-products/zero_hi...,Zero History,NaN,NaN,Под заказ,871,NaN,[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,Zero History - a gripping technothriller from ...,9780670919550,...,NaN,NaN,287,198(д) х 129(ш) х 25(в),NaN,NaN,NaN,NaN,NaN,NaN
4,https://shop.relod.ru/catalog-products/yours_f...,Yours Forever,NaN,NaN,Под заказ,1113,NaN,[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,"For fans of J Lynn, Jodi Ellen Malpas and Sylv...",9780349407210,...,NaN,NaN,172,199(д) х 127(ш) х 21(в),NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,https://shop.relod.ru/catalog-products/the_cas...,The Case-Book of Sherlock Holmes,NaN,NaN,Под заказ,582,NaN,[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,In The Case-Book of Sherlock Holmes we read th...,9780199555642,...,Соединенное Королевство,NaN,236,197(д) х 131(ш) х 16(в),NaN,NaN,NaN,NaN,NaN,NaN
563,https://shop.relod.ru/catalog-products/the_cas...,The Case of the General's Thumb,NaN,NaN,Под заказ,871,NaN,[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,When the corpse of a distinguished general and...,9780099455257,...,NaN,NaN,138,198(д) х 129(ш) х 13(в),NaN,NaN,NaN,NaN,NaN,NaN
564,https://shop.relod.ru/catalog-products/the_car...,The Careful Use of Compliments,NaN,NaN,Под заказ,1113,NaN,[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,"For philosophically minded Isabel Dalhousie, e...",9780349139432,...,NaN,NaN,179,197(д) х 131(ш) х 17(в),NaN,NaN,NaN,NaN,NaN,NaN
565,https://shop.relod.ru/catalog-products/the_can...,The Canterbury Tales,NaN,NaN,Под заказ,968,NaN,[https//opt-1458870.ssl.1c-bitrix-cdn.ru/uploa...,"""The Canterbury Tales"" is a major part of Engl...",9780141442297,...,NaN,NaN,321,198(д) х 129(ш) х 20(в),NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df.to_csv('hw_3.csv', index=False)